<a href="https://colab.research.google.com/github/imdeepmind/UdacitySelfDrivingCar/blob/master/Base_Model_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Getting the into colab
!git clone https://github.com/imdeepmind/UdacitySelfDrivingCar.git
!mkdir data
!mv UdacitySelfDrivingCar/* ./
!rm -r UdacitySelfDrivingCar

Cloning into 'UdacitySelfDrivingCar'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 5227 (delta 14), reused 10 (delta 5), pack-reused 5200
Receiving objects: 100% (5227/5227), 66.03 MiB | 31.10 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [16]:
# Dependencies
import pandas as pd
import numpy as np
import cv2
import torch

from torch.utils.data import Dataset, DataLoader, sampler
import torchvision.transforms as transforms


In [35]:
# Constants
CSV_FILE = './data/driver_log_cleaned.csv'
BATCH_SIZE=32

In [9]:
# Reading CSV file
data = pd.read_csv(CSV_FILE)
data.head()

,center_camera,left_camera,right_camera,steering,throttle,reverse,speed
0,./data/IMG/center_2020_09_05_16_40_17_113.jpg,./data/IMG/left_2020_09_05_16_40_17_113.jpg,./data/IMG/right_2020_09_05_16_40_17_113.jpg,0.0,0.0,0,0.000077
1,./data/IMG/center_2020_09_05_16_40_17_214.jpg,./data/IMG/left_2020_09_05_16_40_17_214.jpg,./data/IMG/right_2020_09_05_16_40_17_214.jpg,0.0,0.0,0,0.000077
2,./data/IMG/center_2020_09_05_16_40_17_320.jpg,./data/IMG/left_2020_09_05_16_40_17_320.jpg,./data/IMG/right_2020_09_05_16_40_17_320.jpg,0.0,0.0,0,0.000077
3,./data/IMG/center_2020_09_05_16_40_17_425.jpg,./data/IMG/left_2020_09_05_16_40_17_425.jpg,./data/IMG/right_2020_09_05_16_40_17_425.jpg,0.0,0.0,0,0.000077
4,./data/IMG/center_2020_09_05_16_40_17_531.jpg,./data/IMG/left_2020_09_05_16_40_17_531.jpg,./data/IMG/right_2020_09_05_16_40_17_531.jpg,0.0,0.0,0,0.000077


In [15]:
left_image = data["left_camera"].values
center_image = data["center_camera"].values
right_image = data["right_camera"].values

steering = data["steering"].values

In [14]:
# Custom Data Loader class
class DriveDataset(Dataset):
  def __init__(self, left_images, center_images, right_images, steering, transform=None):
    self.__left_images = left_images
    self.__center_images = center_images
    self.__right_images = right_images

    self.__steering = steering

    self.__transform = transform
  
  def __len__(self):
    return len(self.__steering)
  
  def __getitem__(self, index):
    left = cv2.imread(self.__left_images[index], 1)
    center = cv2.imread(self.__center_images[index], 1)
    right = cv2.imread(self.__right_images[index], 1)

    stacked = np.hstack([left, center, right])

    if self.__transform:
      stacked = self.__transform(stacked)

    steering =  torch.tensor(float(self.__steering[index]))

    return stacked, steering

In [34]:
# Making the data loader generator
dataset = DriveDataset(left_image, center_image, right_image, steering, transform=transforms.ToTensor())
train_set, test_set = torch.utils.data.random_split(dataset, [int(len(left_image) * .8), int(len(steering) - (len(steering) * .8) + 1) ])

train_loader = DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=True)

In [44]:
# Testing the loader
for i, (images, labels) in enumerate(train_loader):
    print(images.shape, labels.shape)
    break

torch.Size([32, 3, 160, 960]) torch.Size([32])
